<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=68f899dce3c1a970aca7ea6fadf2476358e910973b792674d954e1c42aa3cb48
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-24 13:46:34
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -55.92 K (-0.39%)
Current PnL: -23.18 L (-15.19%)
CY Booked + Current PnL: -9.29 L (-6.09%)
-------------------
Total profit:  1.55 L
Total loss:  -24.73 L
-------------------
Total Booked + Current PnL: 17.69 L (14.08%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.74%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.59 L (62.12%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.23%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,0.05,-14.19,16.65,0.09,14398.0,-14304.0,86473.0,100.64,49.0,M-SC,4.25,253.0,-0.99,0.61,13.61,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.0,0.22,-8.79,9.65,0.02,15365.0,-15337.0,159222.0,142.75,71.0,M-SC,5.46,234.0,-1.00,1.12,38.50,OX40N,NTT,PAINTS
43,ITC,452.0,-0.70,-1.05,11.60,10.44,22973.0,-2093.0,198045.0,-40.74,45.0,X-LC,0.77,5.0,-0.09,1.39,4.46,X40,NTT,FMCG
53,NESTLEIND,1377.0,-0.30,12.56,7.85,21.40,23454.0,33345.0,298771.0,4.42,60.0,X-LC,7.93,12.0,1.42,2.10,20.04,XY25,NTT,FMCG
83,VOLTAS,1530.0,-1.63,7.41,11.43,19.69,23540.0,14208.0,205950.0,-3.01,49.0,X-MC,3.54,78.0,0.60,1.44,14.71,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,3.10,-27.11,95.98,42.84,45476.0,-17625.0,47381.0,-52.07,42.0,X-SC,37.19,83.0,-0.39,0.33,3.10,XY24,NTT,MISC
34,HONAUT,58357.33,2.81,-13.81,66.28,43.32,93044.0,-22492.0,140380.0,-25.97,41.0,X-SC,1.93,90.0,-0.24,0.98,7.79,X40N,ATH,ELECTRICAL
70,SURYODAY,216.00,2.10,-17.97,49.99,23.03,73430.0,-32181.0,146890.0,60.09,52.0,H-SC,11.22,167.0,-0.44,1.03,45.74,XR,NTT,BANKS
26,GILLETTE,11206.78,2.07,-0.18,32.66,32.42,82774.0,-456.0,253440.0,-17.85,38.0,X-SC,6.62,85.0,-0.01,1.78,14.89,X40,ATH,FMCG
29,HAPPSTMNDS,1480.71,1.65,-36.94,193.15,84.85,156096.0,-47349.0,80816.0,-25.05,48.0,H-SC,14.25,147.0,-0.30,0.57,3.72,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.00,-3.78,-46.29,224.24,74.15,197046.0,-75732.0,87873.0,-257.83,25.0,H-SC,14.60,149.0,-0.38,0.62,12.64,XR,NTT,CHEMICALS
72,TANLA,1943.92,-3.45,-36.97,248.56,119.69,424183.0,-100105.0,170656.0,-36.34,26.0,H-SC,12.00,156.0,-0.24,1.20,34.82,AR,ATH,IT
27,GLAXO,3466.20,-3.38,-3.11,43.62,39.15,84221.0,-6204.0,193080.0,-22.20,26.0,X-MC,9.11,60.0,-0.07,1.35,24.99,X40,ATH,PHARMA
58,RAJOOENG,143.10,-3.16,-29.85,99.03,39.61,71203.0,-30600.0,71900.0,-52.57,20.0,H-SC,19.28,135.0,-0.43,0.50,0.00,AR,ATH,MISC
21,DEN,75.00,-2.91,-42.49,147.04,42.07,71426.0,-35888.0,48576.0,73.58,25.0,M-SC,7.34,235.0,-0.50,0.34,3.16,AR,NTT,ENTERTAINMENT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.41,-0.79,108.37,106.72,132603.0,-975.0,122361.0,-50.82,32.0,H-SC,4.06,139.0,-0.01,0.86,22.4,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.40,-5.32,119.61,107.91,167193.0,-7862.0,139782.0,-22.98,32.0,M-SC,10.37,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.22,-8.79,9.65,0.02,15365.0,-15337.0,159222.0,142.75,71.0,M-SC,5.46,234.0,-1.00,1.12,38.50,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.05,-14.19,16.65,0.09,14398.0,-14304.0,86473.0,100.64,49.0,M-SC,4.25,253.0,-0.99,0.61,13.61,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.37,-22.59,46.58,13.47,97238.0,-60912.0,208755.0,-68.12,27.0,H-SC,1.52,158.0,-0.63,1.46,6.15,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,0.46,-22.97,71.39,32.03,72500.0,-30280.0,101555.0,-16.71,48.0,H-SC,6.64,148.0,-0.42,0.71,13.75,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.23,1.52,69.69,72.27,119880.0,2571.0,172019.0,-12.89,57.0,M-LC,4.61,99.0,0.02,1.21,7.79,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.23,1.52,69.69,72.27,119880.0,2571.0,172019.0,-12.89,57.0,M-LC,4.61,99.0,0.02,1.21,7.79,XR,NTT,IT
38,INDIAMART,4810.62,-1.41,-0.79,108.37,106.72,132603.0,-975.0,122361.0,-50.82,32.0,H-SC,4.06,139.0,-0.01,0.86,22.40,AR,ATH,MISC
25,FINCABLES,1641.55,-1.40,-5.32,119.61,107.91,167193.0,-7862.0,139782.0,-22.98,32.0,M-SC,10.37,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,0.19,-2.35,38.89,35.63,79591.0,-4917.0,204657.0,-16.30,33.0,H-MC,4.51,119.0,-0.06,1.44,13.93,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,-0.01,-5.42,92.34,81.92,100261.0,-6220.0,108578.0,-44.55,31.0,M-SC,5.32,236.0,-0.06,0.76,28.54,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JSWINFRA,342.0,-1.07,-10.04,28.09,15.23,50625.0,-20122.0,180225.0,-22.98,23.0,X-MC,6.51,66.0,-0.40,1.26,18.97,X40N,NTT,REALTY
76,TMPV,600.0,-0.77,-14.82,66.92,42.18,157556.0,-40970.0,235440.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
27,GLAXO,3466.2,-3.38,-3.11,43.62,39.15,84221.0,-6204.0,193080.0,-22.20,26.0,X-MC,9.11,60.0,-0.07,1.35,24.99,X40,ATH,PHARMA
11,BATAINDIA,2096.0,0.72,-35.00,108.02,35.20,90338.0,-45039.0,83631.0,9.60,26.0,X-SC,12.61,92.0,-0.50,0.59,0.72,X40,NTT,FOOTWEAR
60,RELAXO,1176.0,-0.96,-47.61,190.69,52.31,145029.0,-69105.0,76055.0,-44.25,28.0,X-SC,3.79,91.0,-0.48,0.53,1.80,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-0.77,-14.82,66.92,42.18,157556.0,-40970.0,235440.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.70,-1.05,11.60,10.44,22973.0,-2093.0,198045.0,-40.74,45.0,X-LC,0.77,5.0,-0.09,1.39,4.46,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.55,1.83,28.61,30.97,51491.0,3237.0,179975.0,-22.18,51.0,X-MC,1.55,75.0,0.06,1.26,14.62,X40,ATH,INSURANCE
3,ACC,3906.0,-0.52,-23.49,114.56,64.15,208556.0,-55901.0,182050.0,-55.44,41.0,X-SC,1.91,82.0,-0.27,1.28,2.12,XY24,BTT,CEMENT
66,SIEMENS,4671.5,0.14,-14.61,46.99,25.51,74667.0,-27195.0,158900.0,-2.45,54.0,H-LC,1.93,49.0,-0.36,1.11,17.10,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-0.77,-14.82,66.92,42.18,157556.0,-40970.0,235440.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.17,-13.54,71.70,48.45,124302.0,-27142.0,173364.0,-19.66,31.0,X-MC,7.47,64.0,-0.22,1.22,0.17,X40N,ATH,FINANCE
54,PAGEIND,51605.08,-0.13,-5.13,32.88,26.07,51076.0,-8400.0,155340.0,-30.18,35.0,X-MC,8.87,55.0,-0.16,1.09,0.70,X40,ATH,APPARELS
11,BATAINDIA,2096.00,0.72,-35.00,108.02,35.20,90338.0,-45039.0,83631.0,9.60,26.0,X-SC,12.61,92.0,-0.50,0.59,0.72,X40,NTT,FOOTWEAR
55,PGHH,17907.65,-0.21,-4.22,39.65,33.76,76265.0,-8475.0,192345.0,-32.38,34.0,X-MC,3.20,57.0,-0.11,1.35,0.76,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,3.10,-27.11,95.98,42.84,45476.0,-17625.0,47381.0,-52.07,42.0,X-SC,37.19,83.0,-0.39,0.33,3.10,XY24,NTT,MISC
60,RELAXO,1176.00,-0.96,-47.61,190.69,52.31,145029.0,-69105.0,76055.0,-44.25,28.0,X-SC,3.79,91.0,-0.48,0.53,1.80,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.72,-35.00,108.02,35.20,90338.0,-45039.0,83631.0,9.60,26.0,X-SC,12.61,92.0,-0.50,0.59,0.72,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.32,2.77,20.73,24.08,27045.0,3515.0,130465.0,-5.49,44.0,X-SC,4.74,89.0,0.13,0.91,22.93,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-1.10,-4.95,27.92,21.58,37189.0,-6939.0,133200.0,-52.09,33.0,X-MC,7.02,56.0,-0.19,0.93,17.92,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.64,-13.84,38.45,19.28,114606.0,-47891.0,298065.0,-24.92,66.0,X-LC,7.62,1.0,-0.42,2.09,10.19,X40,ATH,IT
41,INFY,2275.00,0.96,6.19,45.84,54.87,152308.0,19371.0,332259.0,-17.44,63.0,X-LC,3.16,2.0,0.13,2.33,14.99,X40,BTT,IT
76,TMPV,600.00,-0.77,-14.82,66.92,42.18,157556.0,-40970.0,235440.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
82,VBL,671.64,-0.09,-9.66,50.17,35.67,143158.0,-30496.0,285346.0,-20.46,41.0,X-LC,6.05,4.0,-0.21,2.00,2.86,X40N,ATH,FMCG
43,ITC,452.00,-0.70,-1.05,11.60,10.44,22973.0,-2093.0,198045.0,-40.74,45.0,X-LC,0.77,5.0,-0.09,1.39,4.46,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-2.66,-15.08,113.93,81.67,90763.0,-14144.0,79666.0,7161.11,45.0,L-SC,18.46,271.0,-0.16,0.56,54.28,XR,NTT,CERAMICS
68,SONACOMS,806.63,1.13,-11.97,58.52,39.54,52111.0,-12110.0,89049.0,-31.16,69.0,M-SC,6.85,220.0,-0.23,0.62,25.57,AR,ATH,AUTO
51,MASFIN,398.61,-1.99,-4.85,28.27,22.05,26355.0,-4755.0,93225.0,-17.80,51.0,H-SC,7.22,164.0,-0.18,0.65,35.51,XR,ATH,FINANCE
70,SURYODAY,216.00,2.10,-17.97,49.99,23.03,73430.0,-32181.0,146890.0,60.09,52.0,H-SC,11.22,167.0,-0.44,1.03,45.74,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.22,-8.79,9.65,0.02,15365.0,-15337.0,159222.0,142.75,71.0,M-SC,5.46,234.0,-1.00,1.12,38.50,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.22,-8.79,9.65,0.02,15365.0,-15337.0,159222.0,142.75,71.0,M-SC,5.46,234.0,-1.00,1.12,38.50,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.16,0.71,20.09,20.95,50939.0,1786.0,253554.0,-0.35,76.0,X-LC,12.74,31.0,0.04,1.78,36.60,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,-0.09,-37.56,112.82,32.89,54392.0,-28996.0,48211.0,-695.06,63.0,L-MC,5.61,259.0,-0.53,0.34,32.77,XR,NTT,BANKS
50,LTIM,7201.88,0.59,8.98,20.82,31.66,55849.0,22094.0,268245.0,4.85,63.0,H-LC,12.51,43.0,0.40,1.88,48.84,X200,ATH,IT
0,5PAISA,593.00,-2.26,-36.20,76.96,12.90,107284.0,-79094.0,139402.0,105.53,56.0,H-SC,14.57,161.0,-0.74,0.98,15.91,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.83
1,25,44.72
2,50,75.50


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.53
MC    29.25
LC    25.22
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.87
X40      21.25
X40N     11.87
XR        9.64
XY25      9.10
AR        8.95
OX40N     7.69
X200      1.88
SR        0.98
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.89
X-MC    22.83
X-LC    20.05
M-SC    12.04
X-SC     8.22
H-MC     4.71
H-LC     2.99
L-SC     1.38
M-MC     1.37
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.82,-4.94,38.88
IT,13.86,-14.41,73.47
FINANCE,9.41,-15.59,64.44
MISC,7.40,-22.86,73.20
PAINTS,6.01,-7.77,23.77
ELECTRICAL,5.91,-11.24,51.99
INSURANCE,4.41,-0.45,35.85
PHARMA,3.83,-2.30,35.04
AUTO,3.36,-19.91,71.33


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3062548.0,21
AR,1296411.0,10
XR,1287561.0,13
X40,1008598.0,14
X40N,754052.0,9
OX40N,696028.0,10
XY25,343984.0,6
SR,280149.0,2
MH,74003.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3572506.0,25
M-SC,1343934.0,15
X-MC,1224421.0,16
X-LC,904533.0,11
X-SC,746551.0,8
H-MC,407394.0,3
L-SC,263800.0,3
H-LC,130516.0,2
M-LC,119880.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1225464.0      6
           AR         881453.0      5
           XR         788493.0      7
M-SC       XY24       786317.0      6
X-MC       X40        448003.0      7
X-LC       X40        387483.0      5
X-MC       XY24       348263.0      3
H-SC       OX40N      322944.0      4
X-SC       X40N       292362.0      3
M-SC       OX40N      284665.0      5
X-SC       XY24       281077.0      3
H-SC       SR         280149.0      2
X-MC       X40N       248569.0      4
X-LC       XY24       230787.0      2
H-MC       AR         216754.0      2
X-LC       X40N       213121.0      2
H-MC       XY24       190640.0      1
X-MC       XY25       179586.0      2
L-SC       XR         175381.0      2
X-SC       X40        173112.0      2
M-SC       XR         149415.0      2
           AR         123537.0      2
M-LC       XR         119880.0      1
L-SC       OX40N       88419.0      1
H-LC       AR          74667.0      1
H-SC       MH          74003.0      1
X-LC       XY25        73142.0      2
H-LC       X200        55849.0      1
L-MC       XR          54392.0      1
M-MC       XY25        50030.0      1
L-LC       XY25        41226.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
